In [27]:
from ase.io import read,write
import cace
evaluator = cace.tasks.EvaluateTask(model_path='../fit-4hdnnp-NaCl/loss_data/Nacl_model.pth',
#model_path='/dssg/home/acct-matxzl/matxzl/Yajie/MDNN/CACE-LR-4090/cace-lr-fit-main/fit-4hdnnp-Au2-MgO-Al/loss_data/old/dl_1_loss20250327_102251_model.pth', device='cpu',
energy_key='CACE_energy', #'ewald_potential',
forces_key='CACE_forces',
#other_keys='q',
)

data = read('../fit-4hdnnp-NaCl/NaCl.xyz',':')

pre = evaluator(data)
pre['energy']

CUDA version: 12.1, CUDA device: 0


array([ 0.15400946,  0.0252136 ,  0.07934177, ...,  0.07319129,
       -0.02893162, -0.16877043], dtype=float32)

In [28]:
import numpy as np
def get_property(atoms, info_name, atomic_energies=None):
    if info_name == 'energy' and atomic_energies is not None:
        # 对于每个结构，计算总能量并扣除基态能量
        ene_results = []
        num_results = []
        for a in atoms:
            energy = a.info.get(info_name, None)
            if energy is None:
                raise ValueError(f"Property '{info_name}' not found in atoms info.")
            # 获取原子序数
            atomic_numbers = a.get_atomic_numbers()
            # 扣除基态能量
            energy -= sum(atomic_energies.get(Z, 0) for Z in atomic_numbers)
            ene_results.append(energy)
            num_results.append(len(atomic_numbers))
        return np.array(ene_results),np.array(num_results)

In [29]:
atomic_energies={11: -4417.07609365649, 17: -12516.880649933015}
ref_energy,atom_num= get_property(data, 'energy', atomic_energies=atomic_energies)

In [30]:
pre['energy']

array([ 0.15400946,  0.0252136 ,  0.07934177, ...,  0.07319129,
       -0.02893162, -0.16877043], dtype=float32)

In [31]:
ref_energy

array([ 0.15517016,  0.02694292,  0.07818196, ...,  0.07356963,
       -0.02910438, -0.16642664])

In [32]:
np.sqrt(np.mean(((ref_energy-pre['energy'])/atom_num)**2))

0.0001433191843173955

In [33]:
def get_forces(atoms_list):
    forces_list = []
    for a in atoms_list:
        # 检查是否有力
        if 'forces' in a.arrays:
            forces_list.append(a.arrays['forces'])
        else:
            # 如果没有，可以用None或者空数组，但这里我们选择用None表示缺失
            forces_list.append(None)
    return forces_list

In [34]:
ref_forces = get_forces(data)

In [35]:
ref_force = np.concatenate(ref_forces, axis=0)

In [36]:
np.sqrt(np.mean((ref_force-pre['forces'])**2))

0.004016742096638881